# FIT5196 Assessment 1
## TASK 2: Text Pre-processing
#### Student Name: Harshavardhan Reddy Mallypally
#### Student ID: 29424143

Date: 12/08/2018
Juypter version 5.5.0

Environment: Python 3.6.0 and Anaconda 5.2.0 (64-bit)

Packages used:             
                **regular expression**        
                **nltk**               
                **itertools**

# 1.Introduction
This task is of analyzing textUal data, i.e., converting the extracted data into a proper format. Here, we written python code  to preprocess a set of resumes that are provided and convert them into numerical representations.

Since, I can't run **pdfminer** I'm going to use text format resumes that shared in drive. 

# 2. Import package

In [ ]:
import re
import nltk
from nltk.collections import *
import itertools
from itertools import chain

# 3. Extracting Resumes
## Approach to extract the resumes
1. First of all, extracted the file numbers under my student ID from the <font color="blue">resume_dataset.txt</font> file.
2. Using file numbers open the resume files and regex <font color="blue">r"\w+(?:[-']\w+)?"</font> given in the assignment extracted text from ach resume.

In [ ]:
# opening resume_dataset.txt for extracting file_numbers
with open('resume_dataset.txt','r') as myresume:
    resume = myresume.read()
    resume_num = re.findall(r"29424143:\S(.*?)(?s)[]]",resume)
temp1=(str(resume_num).replace('\\n','')).replace(' ','*')
file_numbers = re.findall(r'(\d{1,3})',str(temp1))
myresume.close()

# using given regex, resume files words are extracted 
token_vocab=[]
for i in range(0,len(file_numbers)):
    filename='resume_('+str(file_numbers[i])+').txt'
    with open(filename,'r',encoding="utf-8") as file:
        text = file.read()
        vocab_regex = re.findall(r"\w+(?:[-']\w+)?",text)
        token_vocab.append(vocab_regex)

# 4. Porter Stemmer

Stemmer is used to find the stem word for each and every word from resume files

In [ ]:
#stemmer is implimented on tokens
stemmer = nltk.PorterStemmer()
stemmed_list = []
for filetxt in token_vocab:
    wordlist = []
    for word in filetxt:
        new_word = stemmer.stem(word)
        wordlist.append(word)
    stemmed_list.append(wordlist)

## 5. Stop Words Removal

stopwords are given in stopwords_en.txt to remove stopwords for the words extracted for the resume files

In [ ]:
# stopwords are extracted
with open("stopwords_en.txt","r") as stopwd:
    stopwords = stopwd.read()
stopword = stopwords.split("\n")


# stopwords are remove for resume words
temp=[]
stop_words=[]
stop_rm_vocab=[]
for stem_words in stemmed_list:
    for word in stem_words:
        if word not in stopword:
            stop_words.append(word)
            temp.append(word)
    stop_rm_vocab.append(stop_words)
    stop_words=[]

## 6. Removing Context Dependent and Context Independent 

removing the 98% threshold words and 2% rare words for creating the final vocabulary list

In [ ]:
# duplicates are removed for the resume words 
total_words= []
for filetxt in stop_rm_vocab:
    for word in filetxt:
        if word not in total_words:
            total_words.append(word)
            
# (98% and above) and (2& and below) threshold worlds are removed
indep_word  = []
for total in total_words:
    count = 0
    for stop_rm_vocab1 in stop_rm_vocab:
        if total in stop_rm_vocab1:
            count += 1
    if count <245 and count>5:
        indep_word.append(total)

# 7. Finding Bigrams
Finding  the Bigrams using ntlk packages 

In [ ]:
# 200 bigrams are found
lists=  []
for st in stop_rm_vocab:
    lists += st


bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(st)
bigram_finder.apply_freq_filter(0)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200)

# 8. Creating vocab text
Creating vocab text containing both bigrams and unigrams

In [ ]:
#vocab text is created to load bigrams and vocab words 
voc_file = open("29414243_vocab.txt","w",encoding='utf-8')

#top 200 bigrams are added here
voc_string = "**********************************200 MEANINGFUL BIGRAMS*************************************"
voc_file.write(voc_string)
for tuples in top_200_bigrams:
    key,value = tuples
    voc_file.write(str(key)+','+str(value)+'\n')
    
# vocabs are added from here
voc_string = "**********************************VOCAB*************************************"
for word in indep_word:
    voc_file.write(str(word)+':'+str(indep_word.index(word))+'\n')
voc_file.close()

# 9. Sparse representation
Sparse Representation of words which containes the filename follwed by token index followed by the count

In [ ]:
# empty list created
countvec=[]
# for loop is created to get the filname, token_index and count
for filetxt in stop_rm_vocab:
    count_voc = open("29424143_countvec.txt","w") 
    count=0
    for word in indep_word:
        tok_index= indep_word.index(word)
        if word in filetxt:
            count1 = filetxt.count(word)
            countvec.append('resume_('+str(file_numbers[stop_rm_vocab.index(filetxt)])+').txt:'+str(tok_index)+':'+str(count1))    
#countvect.txt file is opened in write mode
count_voc = open("29424143_countvec.txt","w")    
count_voc.write("\n".join(countvec))
count_voc.close()

# 10. Summary

\* resume files are extracted                                        
\* stemmer applied on the words                         
\* Stop words are removed                                   
\* context dependency is also removed                        
\* top 200 bigrams are found                            
\* created vocab_text                                        
\* created Sparse representation                                        